In [48]:
import numpy as np
import pandas as pd

In [97]:
catalog = "../../Bases_de_datos/Data_catalog.csv"
iso = "../../Bases_de_datos/Country_ISO_code.csv"
path_v = "../data/variables.csv"
ix = "alpha-3"

df_c = pd.read_csv(catalog)
df_iso = pd.read_csv(iso)
data_0 = pd.DataFrame(index = df_iso[ix])
data_0.index = data_0.index.set_names(ix)
data_s   = data_0.copy()
data_c   = data_0.copy()
data_g   = data_0.copy()
data_q_s = data_0.copy()
data_q_c = data_0.copy()
data_q_g = data_0.copy()

df_v = pd.read_csv(path_v)

df_c[ df_c["Origin"] == "UNESCO UIS" ]


,Dataset,ID,Origin,Countries,Granularity,Timestep,Annex,Website,Filename,Path
20,Demographic and Socio-economic Indicators,DEM,UNESCO UIS,All,National,Yearly,NaN,https://uis.unesco.org/bdds,DEM_DATA_NATIONAL.csv,Bases_de_datos/UNESCO_UIS/DEM/
21,Other Policy Relevant Indicators (OPRI),OPRI,UNESCO UIS,All,National,Yearly,NaN,https://uis.unesco.org/bdds,OPRI_DATA_NATIONAL.csv,Bases_de_datos/UNESCO_UIS/OPRI/
22,Research and Development (R&D) – Other Policy ...,SCN-OPRI,UNESCO UIS,All,National,Yearly,NaN,https://uis.unesco.org/bdds,SCN-OPRI_DATA_NATIONAL.csv,Bases_de_datos/UNESCO_UIS/SCN-OPRI/
23,Research and Development (R&D) SDG 9.5,SCN-SDG,UNESCO UIS,All,National,Yearly,NaN,https://uis.unesco.org/bdds,SCN-SDG_DATA_NATIONAL.csv,Bases_de_datos/UNESCO_UIS/SCN-SDG/
24,SDG Global and Thematic Indicators,SDG,UNESCO UIS,All,National,Yearly,NaN,https://uis.unesco.org/bdds,SDG_DATA_NATIONAL.csv,Bases_de_datos/UNESCO_UIS/SDG/
25,SDG 11.4,SDG11,UNESCO UIS,All,National,Yearly,NaN,https://uis.unesco.org/bdds,SDG11_DATA_NATIONAL.csv,Bases_de_datos/UNESCO_UIS/SDG11/


In [96]:
for row in df_v[ df_v["Category"] == "socioeconomic" ].itertuples():
    v = row.Variable
    if row.Origin == "World Bank": nr = 3
    else: nr = 0
    
    df = pd.read_csv( "../../"
        + df_c.loc[df_c["ID"]==row.ID, 'Path'].iloc[0]
        + df_c.loc[df_c["ID"]==row.ID, "Filename" ].iloc[0],
        skiprows = nr, low_memory = False )
    
    if row.Origin == "OECD":
        df = df[ df["MEASURE"] == v ]
        df = df.rename(columns = {"REF_AREA": ix})
        df = df[ df[ix].isin(df_iso[ix]) ]
        
        if row.ID in ["DF_GREEN_GROWTH", "DF_NECRS", "DF_REVPOT"]:
            df_i = df[ (df["MEASURE"]==v) ]
            if v == "ECRATE":
                df_i = df_i[ (df["SECTOR"]==row.SECTOR)
                & (df["EMISSIONS_SOURCE"]==row.EMISSIONS_SOURCE) ]
            data_g[v] = df_i.sort_values( [ix, "TIME_PERIOD"]
                ).drop_duplicates(ix, keep="last").set_index(ix)["OBS_VALUE"]
            if v == "POP_NETMIGR": data_g[v] = np.abs( data_g[v] )
            
    elif row.Origin == "Global Data Lab":
        df = df.rename(columns = {"ISO_Code": ix})
        df = df[ df[ix].isin(df_iso[ix]) ]
        data_s[v] = df[ df["Level"] == "National" ].set_index(ix)[v]

    elif row.Origin == "World Bank":
        df = df.rename(columns = {"Country Code": ix})
        df = df[ df[ix].isin(df_iso[ix]) ]
        data_s[v] = df.set_index(ix).iloc[:, 4:-1].ffill(axis = 1).iloc[:, -1]

    elif row.Origin == "UNESCO UIS":
        if row.ID in ["DEM", "SCN-SDG"]:
            df.columns = [ x.lower() for x in df.columns ]
        df = df.rename(columns = {"country_id": ix})
        df = df[ df[ix].isin(df_iso[ix]) ]
        if row.PERCENTAGE == "Yes": pop = data_s["200101"]
        else: pop = 1
        data_s[v] = ( df[ df["indicator_id"]==v ].sort_values( [ix, "year"]
            ).drop_duplicates(ix, keep="last").set_index(ix)["value"] / pop )
    
    # corregir random
    data_q_s[v] = ( 1 + pd.qcut( data_s[v]
        + np.random.random(data_s[v].shape[0])/10000, 20, labels = False ) )
    if row.Negative_indicator:
        data_q_s[v] = 21 - data_q_s[v]
        
data_q_s = data_q_s / 2
data_q_s["Mean"] = data_q_s.mean(axis = 1)
data_q_s["Count"] = data_q_s.count(axis = 1)
data_q_s = data_q_s[ data_q_s["Count"] > 1 ].sort_values("Mean")

data_q_s

KeyError: 'POP_NETMIGR'

In [98]:
for cat in df_v["Category"].unique():
    data   = data_0.copy()
    data_q = data_0.copy()

    for row in df_v[ (df_v["Category"] == cat) ].itertuples():

        print(row)
        v = row.Variable
        if row.Origin == "World Bank": nr = 3
        else: nr = 0
        
        df = pd.read_csv( "../../"
            + df_c.loc[df_c["ID"]==row.ID, 'Path'].iloc[0]
            + df_c.loc[df_c["ID"]==row.ID, "Filename" ].iloc[0],
            skiprows = nr, low_memory = False )
        
        if row.Origin == "OECD":
            df = df[ df["MEASURE"] == v ]
            df = df.rename(columns = {"REF_AREA": ix})
            df = df[ df[ix].isin(df_iso["alpha-3"]) ]
            
            df_i = df[ (df["MEASURE"]==v) ]
            
            if v == "POP_NETMIGR": df_i["OBS_VALUE"] = np.abs( df_i["OBS_VALUE"] )
            
            elif v in ["HD_POP_EXP", "HD_TN_POP_EXP", "ID_POP_EXP",
                "99PT_LAND_EXP", "99PT_CROP_EXP", "EP_LAND_EXP"]:
                df_i = df_i[ (df_i["DURATION"]==row.DURATION) ]
            elif v in ["TEMP_CHANGE", "ED_CROP_ANOM", "EP_CHANGE"]:
                df_i = df_i.set_index(ix)
                df_i["OBS_VALUE"] = df_i.reset_index()[
                    [ix, "OBS_VALUE"] ].groupby(ix).mean()
                df_i = df_i.reset_index()
            elif v == "UTCI_POP_EXP":
                df_i = df_i[ (df_i["HEAT_STRESS"]==row.HEAT_STRESS) ]
            elif v in ["CF_BUILT_EXP", "CF_POP_EXP", "CF_CROP_EXP",
                "RF_BUILT_EXP", "RF_POP_EXP", "RF_CROP_EXP"]:
                df_i = df_i[ (df_i["RET_PERIOD"]==row.RET_PERIOD) ]
            elif v == "BURN_LAND_AREA":
                df_i = df_i[ (df_i["UNIT_MEASURE"]==row.UNIT_MEASURE) ]
            elif v in ["CY_BUILT_EXP", "CY_POP_EXP"]:
                df_i = df_i[
                    (df["HURRICANE_WIND_SCALE"]==row.HURRICANE_WIND_SCALE) ]
            elif v == "POP_EXP_POL":
                df_i = df_i[ (df_i["EXPOSURE_LEVEL"]==row.EXPOSURE_LEVEL) ]

            elif v == "ECRATE":
                df_i = df_i[ (df_i["SECTOR"]==row.SECTOR)
                & (df["EMISSIONS_SOURCE"]==row.EMISSIONS_SOURCE) ]
            elif v in ["GPAT_DE", "WATER_FW"]:
                df_i = df_i[ (df_i["UNIT_MEASURE"]==row.UNIT_MEASURE) ]
            
            data[v] = df_i.sort_values( [ix, "TIME_PERIOD"]
                ).drop_duplicates(ix, keep="last").set_index(ix)["OBS_VALUE"]

        elif row.Origin == "Global Data Lab":
            df = df.rename(columns = {"ISO_Code": ix})
            df = df[ df[ix].isin(df_iso["alpha-3"]) ]
            data[v] = df[ df["Level"] == "National" ].set_index(ix)[v]
        
        elif row.Origin == "World Bank":
            df = df.rename(columns = {"Country Code": ix})
            df = df[ df[ix].isin(df_iso["alpha-3"]) ]
            data[v] = df.set_index(ix).iloc[:, 4:-1].ffill(axis = 1).iloc[:, -1]
        
        elif row.Origin == "UNESCO UIS":
            if row.ID in ["DEM", "SCN-SDG", "SDG11"]:
                df.columns = [ x.lower() for x in df.columns ]
            df = df.rename(columns = {"country_id": ix})
            df = df[ df[ix].isin(df_iso["alpha-3"]) ]
            if row.PERCENTAGE == "Yes": pop = data["200101"]
            else: pop = 1
            data[v] = ( df[ df["indicator_id"]==v ].sort_values( [ix, "year"]
                ).drop_duplicates(ix, keep="last").set_index(ix)["value"] / pop )
        
        # corregir random
        data_q[v] = ( 1 + pd.qcut( data[v]
            + np.random.random(data[v].shape[0])/10000, 20, labels = False ) )
        if row.Negative_indicator:
            data_q[v] = 21 - data_q[v]

    data_q = data_q / 2
    data_q["Mean"] = data_q.mean(axis = 1)
    data_q["Count"] = data_q.count(axis = 1)
    data_q = data_q[ data_q["Count"] > 1 ].sort_values("Mean")

    if   cat == df_v["Category"].unique()[0]:
        data_q_s = data_q.copy()
        data_s   = data.copy()
    elif cat == df_v["Category"].unique()[1]:
        data_q_c = data_q.copy()
        data_c   = data.copy()
    elif cat == df_v["Category"].unique()[2]:
        data_q_g = data_q.copy()
        data_g   = data.copy()

data_q_c

Pandas(Index=0, Number=1, Origin='UNESCO UIS', Category='socioeconomic', ID='DEM', Dataset_name='Demographic and Socio-economic Indicators', Negative_indicator=True, Variable='200101', Name='Total population (thousands)', Note=nan, Unit_of_measure='thousands', DURATION=nan, Duration=nan, HEAT_STRESS=nan, Heat_stress_thresholds=nan, RET_PERIOD=nan, Return_periods=nan, UNIT_MEASURE=nan, EXPOSURE_LEVEL=nan, Exposure_levels=nan, SECTOR=nan, Economic_sector=nan, EMISSIONS_SOURCE=nan, Emissions_source=nan, PERCENTAGE=nan, HURRICANE_WIND_SCALE=nan, _26=nan)
Pandas(Index=1, Number=2, Origin='UNESCO UIS', Category='socioeconomic', ID='DEM', Dataset_name='Demographic and Socio-economic Indicators', Negative_indicator=False, Variable='200144', Name='Population aged 15-24 years (thousands)', Note='modified to % of total population', Unit_of_measure='% of total population', DURATION=nan, Duration=nan, HEAT_STRESS=nan, Heat_stress_thresholds=nan, RET_PERIOD=nan, Return_periods=nan, UNIT_MEASURE=nan,

,HD_POP_EXP,HD_TN_POP_EXP,ID_POP_EXP,UTCI_POP_EXP,TEMP_CHANGE,CF_BUILT_EXP,CF_POP_EXP,CF_CROP_EXP,ED_CROP_ANOM,99PT_LAND_EXP,...,FT_FORE_EXP,CY_BUILT_EXP,CY_POP_EXP,W_BUILT_EXP,W_POP_EXP,MEAN_POP,POP_EXP_POL,EP_LAND_EXP,Mean,Count
alpha-3,,,,,,,,,,,,,,,,,,,,,
EGY,1.0,1.0,2.0,1.5,2.5,1.0,2.0,0.5,1.0,4.5,...,0.5,9.5,4.0,6.5,4.5,0.5,1.0,10.0,3.060000,26
NPL,2.0,2.0,1.5,2.5,8.0,NaN,NaN,NaN,1.0,7.0,...,4.0,2.5,2.5,4.0,3.5,1.0,1.5,2.5,3.090909,23
THA,2.5,2.5,3.5,3.0,6.0,1.0,2.5,2.5,2.5,6.0,...,5.5,1.5,2.5,5.5,6.0,2.0,2.0,2.0,3.100000,26
IND,1.0,1.0,2.0,1.0,7.0,3.0,6.0,3.5,0.5,1.0,...,3.0,6.0,6.5,8.0,7.5,1.0,1.5,3.0,3.120000,26
IRQ,0.5,0.5,9.5,0.5,1.0,3.0,4.5,4.0,1.5,3.5,...,0.5,4.0,5.0,5.0,7.0,1.5,1.5,10.0,3.240000,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BLM,9.0,5.0,7.0,NaN,9.5,NaN,NaN,NaN,NaN,7.5,...,9.0,NaN,NaN,NaN,NaN,NaN,NaN,7.5,8.050000,11
CUW,5.0,9.5,7.0,NaN,9.0,NaN,NaN,NaN,NaN,9.0,...,9.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0,8.050000,11
MSR,10.0,10.0,4.5,8.0,10.0,NaN,9.5,10.0,NaN,5.5,...,10.0,NaN,5.5,NaN,7.0,9.0,7.0,8.0,8.075000,21


In [106]:
df_iso = df_iso.merge(
    data_q_s.loc[ data_q_s["Count"] > 11, "Mean" ], on = "alpha-3"
    ).rename( columns = {"Mean": "Socioeconomic resilience [0-10]"} )
df_iso = df_iso.merge(
    data_q_c.loc[ data_q_c["Count"] > 11, "Mean" ], on = "alpha-3"
    ).rename( columns = {"Mean": "Climate resilience [0-10]"} )
df_iso = df_iso.merge(
    data_q_g.loc[ data_q_g["Count"] > 11, "Mean" ], on = "alpha-3"
    ).rename( columns = {"Mean": "Governance resilience [0-10]"} )
df_iso

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,...,BRICS,Socioeconomic resilience [0-10],Climate resilience [0-10],Socioeconomic resilience [0-10],Climate resilience [0-10],Socioeconomic resilience [0-10],Climate resilience [0-10],Socioeconomic resilience [0-10],Climate resilience [0-10],Governance resilience [0-10]
0,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,...,NaN,6.000000,4.080000,6.000000,4.080000,6.000000,4.080000,6.000000,4.080000,6.250000
1,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,...,NaN,5.700000,4.680000,5.700000,4.680000,5.700000,4.680000,5.700000,4.680000,5.166667
2,Angola,AO,AGO,24,ISO 3166-2:AO,Africa,Sub-Saharan Africa,Middle Africa,2.0,202.0,...,NaN,2.583333,5.440000,2.583333,5.440000,2.583333,5.440000,2.583333,5.440000,5.000000
3,Argentina,AR,ARG,32,ISO 3166-2:AR,Americas,Latin America and the Caribbean,South America,19.0,419.0,...,NaN,5.617647,3.800000,5.617647,3.800000,5.617647,3.800000,5.617647,3.800000,5.555556
4,Armenia,AM,ARM,51,ISO 3166-2:AM,Asia,Western Asia,NaN,142.0,145.0,...,NaN,6.645833,4.590909,6.645833,4.590909,6.645833,4.590909,6.645833,4.590909,5.461538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,Venezuela (Bolivarian Republic of),VE,VEN,862,ISO 3166-2:VE,Americas,Latin America and the Caribbean,South America,19.0,419.0,...,NaN,5.727273,3.940000,5.727273,3.940000,5.727273,3.940000,5.727273,3.940000,6.083333
131,Viet Nam,VN,VNM,704,ISO 3166-2:VN,Asia,South-eastern Asia,NaN,142.0,35.0,...,NaN,5.312500,3.660000,5.312500,3.660000,5.312500,3.660000,5.312500,3.660000,5.321429
132,Yemen,YE,YEM,887,ISO 3166-2:YE,Asia,Western Asia,NaN,142.0,145.0,...,NaN,3.804348,4.700000,3.804348,4.700000,3.804348,4.700000,3.804348,4.700000,6.409091
133,Zambia,ZM,ZMB,894,ISO 3166-2:ZM,Africa,Sub-Saharan Africa,Eastern Africa,2.0,202.0,...,NaN,3.208333,4.909091,3.208333,4.909091,3.208333,4.909091,3.208333,4.909091,6.000000


In [78]:
data_q_g[ data_q_g["Count"] > 11 ]

,ECRATE,FSUBS,CO2_AIRTRACAP,LCC_CROPTOARTIF,LCC_NATTOCROP,SW_PERMTONOT,CO2_DBEMCAP,GPAT_DE,NRG_INT,LCC_NATGAIN,...,PA_TERRESTRIAL,WATER_FW,WLIFE_MA,"Patent applications, residents",Scientific and technical journal articles,HEXPCSTPPPCAP.CULHER,HEXPCSTPPPCAP.NATHER,EXPGDP.TOT,Mean,Count
country_id,,,,,,,,,,,,,,,,,,,,,
BRN,NaN,NaN,2.0,1.0,6.0,2.5,0.5,8.0,0.5,6.5,...,NaN,NaN,NaN,1.5,5.0,NaN,NaN,4.0,2.961538,14
BHR,NaN,NaN,1.0,0.5,9.0,0.5,NaN,7.5,0.5,7.0,...,NaN,NaN,NaN,2.0,5.5,NaN,NaN,1.5,3.000000,13
TTO,NaN,NaN,3.0,1.0,4.5,9.5,NaN,7.0,0.5,2.0,...,NaN,NaN,NaN,1.5,4.5,NaN,NaN,1.0,3.041667,13
KWT,NaN,NaN,1.5,0.5,7.5,5.5,NaN,7.5,0.5,8.0,...,NaN,NaN,NaN,1.0,6.0,NaN,NaN,1.5,3.375000,13
TKM,NaN,NaN,6.5,3.5,6.5,4.0,NaN,5.5,1.5,7.5,...,1.0,NaN,NaN,5.0,0.5,0.5,NaN,NaN,3.384615,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GHA,8.0,8.0,9.0,3.5,2.5,8.0,NaN,3.5,9.5,7.5,...,NaN,NaN,NaN,3.5,7.0,NaN,NaN,5.0,6.392857,15
MAR,8.5,10.0,5.0,5.0,3.5,4.5,9.0,5.5,8.0,8.0,...,NaN,NaN,NaN,6.5,7.5,NaN,NaN,6.5,6.500000,16
YEM,NaN,NaN,8.5,7.5,7.5,7.5,NaN,3.0,10.0,6.5,...,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,6.590909,12


In [79]:
data_q_s[ data_q_s["Count"] > 11 ]

,200101,200144,200151,DT.TDS.DECT.GN.ZS,NY.GDP.PCAP.CD,SH.DYN.AIDS.ZS,SI.POV.LMIC,SP.DYN.LE00.IN,SP.DYN.TFRT.IN,SP.POP.GROW,...,ROFST.H.1.F,ROFST.H.1.Q1,ROFST.H.1.RUR,XGDP.FSGOV,POP_NETMIGR,Subnational HDI,Subnational GDI,Gini index,Mean,Count
country_id,,,,,,,,,,,,,,,,,,,,,
SSD,4.0,10.0,9.0,NaN,1.5,1.5,0.5,10.0,1.0,3.0,...,0.5,0.5,0.5,0.5,2.5,0.5,0.5,2.0,2.545455,23
AGO,2.0,8.0,9.5,1.0,3.0,2.0,2.5,9.0,0.5,0.5,...,2.0,2.0,1.0,1.5,2.5,2.5,2.5,0.5,2.583333,25
TCD,3.0,8.5,10.0,8.5,1.0,2.5,1.5,10.0,0.5,0.5,...,0.5,0.5,0.5,2.0,5.0,0.5,0.5,4.5,2.680000,26
PNG,4.0,8.5,8.5,1.0,3.0,3.5,1.0,8.0,3.0,2.5,...,1.0,1.5,2.0,0.5,4.5,2.0,2.5,2.5,2.780000,26
CAF,5.0,10.0,10.0,9.5,0.5,1.5,0.5,10.0,0.5,2.0,...,1.0,1.5,1.5,1.0,1.0,0.5,0.5,2.0,2.826087,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LUX,7.5,2.0,3.0,NaN,10.0,6.5,10.0,1.0,9.0,3.0,...,NaN,NaN,NaN,6.5,5.0,9.5,8.5,7.0,7.184211,20
BLR,4.5,1.0,2.5,3.0,5.5,5.0,10.0,5.5,8.5,10.0,...,10.0,10.0,10.0,7.5,4.0,7.0,9.5,10.0,7.260000,26
CUW,8.5,4.5,3.0,NaN,7.0,NaN,NaN,2.5,9.0,10.0,...,NaN,NaN,NaN,9.5,8.5,NaN,NaN,NaN,7.375000,13


In [80]:
data_q_c[ data_q_c["Count"] > 11 ]

,HD_POP_EXP,HD_TN_POP_EXP,ID_POP_EXP,UTCI_POP_EXP,TEMP_CHANGE,CF_BUILT_EXP,CF_POP_EXP,CF_CROP_EXP,ED_CROP_ANOM,99PT_LAND_EXP,...,FT_FORE_EXP,CY_BUILT_EXP,CY_POP_EXP,W_BUILT_EXP,W_POP_EXP,MEAN_POP,POP_EXP_POL,EP_LAND_EXP,Mean,Count
country_id,,,,,,,,,,,,,,,,,,,,,
IND,1.0,1.0,2.0,1.0,7.0,3.0,6.0,3.5,0.5,1.0,...,3.0,10.0,1.5,4.0,8.0,1.0,1.5,3.0,2.940000,26
MMR,1.0,1.0,2.0,3.0,5.0,5.0,3.5,1.5,3.5,2.5,...,3.5,6.5,1.0,8.0,4.0,2.0,2.0,2.5,3.040000,26
EGY,1.0,1.0,6.0,1.5,2.5,1.0,2.0,0.5,1.0,9.5,...,1.0,9.0,0.5,8.5,5.5,0.5,1.0,8.0,3.280000,26
SUR,6.0,8.5,2.0,9.5,10.0,1.0,1.0,0.5,0.5,0.5,...,6.0,0.5,1.5,9.0,4.5,3.0,5.5,1.5,3.300000,26
UZB,1.5,1.5,1.5,1.0,3.0,NaN,NaN,NaN,8.5,1.5,...,3.0,9.0,3.0,4.0,4.5,2.0,2.0,6.0,3.340909,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ATG,7.5,9.5,8.5,8.5,9.5,9.5,6.5,10.0,NaN,7.0,...,6.5,8.0,1.0,6.0,9.5,5.0,7.0,10.0,7.541667,25
CYM,5.5,7.0,10.0,4.5,6.0,9.5,10.0,8.0,NaN,5.0,...,7.5,9.0,10.0,8.0,9.0,8.0,10.0,7.0,7.687500,25
VIR,7.5,9.0,10.0,10.0,9.5,7.5,9.5,8.0,NaN,4.5,...,9.0,5.5,3.0,7.0,9.0,8.5,8.0,10.0,7.791667,25


In [9]:
for row in df_v[ (df_v["Category"] == "governance") ].itertuples():
    print(row)
    v = row.Variable
    if row.Origin == "World Bank": nr = 3
    else: nr = 0
    
    df = pd.read_csv( "../../"
        + df_c.loc[df_c["ID"]==row.ID, 'Path'].iloc[0]
        + df_c.loc[df_c["ID"]==row.ID, "Filename" ].iloc[0],
        skiprows = nr, low_memory = False )
    
    if row.Origin == "OECD":
        df = df[ df["MEASURE"] == v ]
        df = df.rename(columns = {"REF_AREA": ix})
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        
        df_i = df[ (df["MEASURE"]==v) ]
        if v == "ECRATE":
            df_i = df_i[ (df["SECTOR"]==row.SECTOR)
            & (df["EMISSIONS_SOURCE"]==row.EMISSIONS_SOURCE) ]
        data_g[v] = df_i.sort_values( [ix, "TIME_PERIOD"]
            ).drop_duplicates(ix, keep="last").set_index(ix)["OBS_VALUE"]
        if v == "POP_NETMIGR": data_g[v] = np.abs( data_g[v] )
    
    elif row.Origin == "Global Data Lab":
        df = df.rename(columns = {"ISO_Code": ix})
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        data_g[v] = df[ df["Level"] == "National" ].set_index(ix)[v]
    
    elif row.Origin == "World Bank":
        df = df.rename(columns = {"Country Code": ix})
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        data_g[v] = df.set_index(ix).iloc[:, 4:-1].ffill(axis = 1).iloc[:, -1]
    
    elif row.Origin == "UNESCO UIS":
        if row.ID in ["DEM", "SCN-SDG", "SDG11"]:
            df.columns = [ x.lower() for x in df.columns ]
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        if row.PERCENTAGE == "Yes": pop = data_g["200101"]
        else: pop = 1
        data_g[v] = ( df[ df["indicator_id"]==v ].sort_values( [ix, "year"]
            ).drop_duplicates(ix, keep="last").set_index(ix)["value"] / pop )
    
    # corregir random
    data_q_g[v] = ( 1 + pd.qcut( data_g[v]
        + np.random.random(data_g[v].shape[0])/10000, 20, labels = False ) )
    if row.Negative_indicator:
        data_q_g[v] = 21 - data_q_g[v]

data_q_g = data_q_g / 2
data_q_g["Mean"] = data_q_g.mean(axis = 1)
data_q_g["Count"] = data_q_g.count(axis = 1)
data_q_g = data_q_g[ data_q_g["Count"] > 1 ].sort_values("Mean")

data_q_g

Pandas(Index=50, Number=51, Origin='OECD', Category='governance', ID='DF_NECRS', Dataset_name='Net effective carbon rates', Negative_indicator=False, Variable='ECRATE', Name='Effective carbon rate', Note=nan, Unit_of_measure='Euros per tonne of CO2-equivalent', HEAT_STRESS=nan, Heat_stress_thresholds=nan, RET_PERIOD=nan, Return_periods=nan, UNIT_MEASURE=nan, EXPOSURE_LEVEL=nan, Exposure_levels=nan, DURATION=nan, Duration=nan, SECTOR='_T', Economic_sector='Total', EMISSIONS_SOURCE='_T', Emissions_source='All sources', _24=nan, PERCENTAGE=nan)
Pandas(Index=51, Number=52, Origin='OECD', Category='governance', ID='DF_REVPOT', Dataset_name='Net energy tax revenues and reform potential', Negative_indicator=True, Variable='FSUBS', Name='Fossil fuel subsidies', Note=nan, Unit_of_measure='Percentage of GDP', HEAT_STRESS=nan, Heat_stress_thresholds=nan, RET_PERIOD=nan, Return_periods=nan, UNIT_MEASURE=nan, EXPOSURE_LEVEL=nan, Exposure_levels=nan, DURATION=nan, Duration=nan, SECTOR=nan, Economic_

,CO2_AIRTRACAP,LCC_CROPTOARTIF,LCC_NATTOCROP,SW_PERMTONOT,CO2_DBEMCAP,GPAT_DE,NRG_INT,LCC_NATGAIN,PA_MARINE,MWAS_INT,...,WLIFE_MA,"Patent applications, residents",Scientific and technical journal articles,HEXPCSTPPPCAP.CULHER,HEXPCSTPPPCAP.NATHER,EXPGDP.TOT,Mean,Count,ECRATE,FSUBS
country_id,,,,,,,,,,,,,,,,,,,,,
GRD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,NaN,NaN,NaN,1.833333,3,NaN,NaN
SXM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.937500,4,NaN,NaN
GUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.5,2.083333,3,NaN,NaN
LIE,NaN,0.5,4.0,NaN,NaN,3.0,NaN,1.0,NaN,NaN,...,NaN,NaN,2.5,NaN,NaN,NaN,2.164286,7,NaN,NaN
MAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.5,NaN,NaN,NaN,6.0,2.718750,4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWE,9.0,9.0,3.0,7.5,NaN,10.0,7.5,5.0,NaN,NaN,...,NaN,3.0,5.5,NaN,NaN,NaN,6.534965,13,NaN,NaN
GHA,9.0,3.5,2.5,8.0,NaN,9.5,9.5,7.5,NaN,NaN,...,NaN,3.5,7.0,NaN,NaN,5.0,6.582031,16,8.0,8.0
COD,10.0,7.5,5.0,6.0,NaN,NaN,10.0,5.5,NaN,NaN,...,NaN,5.0,4.0,NaN,NaN,5.0,6.734266,13,NaN,NaN
